# Introducción

Este cuaderno documenta la recopilación automatizada de datos sobre apartamentos en arriendo de Ibagué, utilizando técnicas de web scraping. La información obtenida servirá como base para el desarrollo de modelos de machine learning enfocados en la predicción de precios de arriendo, facilitando así el análisis y la toma de decisiones en el mercado inmobiliario.

# Preparación del entorno de trabajo

Se realiza la carga de las librerías necesarias:

In [1]:
# web scraping
import requests
import lxml.html
import json

# procesamiento
import pandas as pd
import math

# herramientas adicionales
import re
import os
from datetime import datetime

Definición de fecha actual para la identificación de las descargas:

In [2]:
fecha_actual = datetime.now().strftime('%Y%m%d')

# Descarga de información

En esta sección se realiza la descarga de información mediante el procedimiento de web scraping. Posteriormente, se realiza la depuración de la información descargada.

Se inicia creando el objeto `enlace`, que contiene el enlace desde el cuál se realizará la predicción.

In [3]:
enlace='https://www.fincaraiz.com.co/arriendo/apartamentos/ibague/tolima'

Se inicia determinando el número de páginas disponibles, lo que sugiere un número aproximado del total de apartamentos en arriendo.

In [4]:
# indicación para guardar el HTML descargado en un archivo
guardar_debug = True

# definición de un agente de usuario que simula a un navegador real
headers = {
    "User-Agent": (
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/126.0.0.0 Safari/537.36"
    )
}

# se realiza la solicitud de la página web 
# (tiempo máximo de 30 segundos)
# si la respuesta tiene un error HTTP (404, 500, etc.),
# raise_for_status() detiene la solicitud
resp = requests.get(enlace, headers=headers, timeout=30)
resp.raise_for_status()

# si guardar_debug es True, se almacena el HTML en un archivo local "debug.html"
if guardar_debug:
    with open("debug.html", "w", encoding="utf-8") as f:
        f.write(resp.text)

# conversión del contenido de la respuesta en un árbol DOM con lxml,
# lo que permite navegar el HTML con expresiones XPath
arbol = lxml.html.fromstring(resp.content)

# uso de regex sobre el HTML para localizar un texto del estilo: "Mostrando 1 - 21 de N".
m = re.search(r"Mostrando\s+\d+\s*-\s*\d+\s*de\s*([\d\.]+)", resp.text, re.IGNORECASE)
if m:
    # Se extrae el número (grupo 1), se eliminan los separadores de miles (puntos)
    # y se convierte a entero.
    numero_apartamentos = int(m.group(1).replace('.', ''))
    print(f"Total de apartamentos: {numero_apartamentos}")
else:
    # Segundo intento (fallback): buscar en el árbol DOM el primer <div>
    # que contenga la palabra "Mostrando". De ese texto se intenta extraer "de N".
    texto_mostrando = arbol.xpath('normalize-space(string(//div[contains(., "Mostrando")][1]))')
    m2 = re.search(r"de\s*([\d\.]+)", texto_mostrando)
    if m2:
        numero_apartamentos = int(m2.group(1).replace('.', ''))
        print(f"Total de apartamentos: {numero_apartamentos}")
    else:
        # Si no se pudo extraer el número con ninguno de los métodos, se lanza un error.
        raise ValueError("No fue posible extraer el total de apartamentos.")

# se cálcula el número de páginas a navegar
numero_paginas = math.ceil(numero_apartamentos / 21)
print(f"Total de páginas: {numero_paginas}")

Total de apartamentos: 167
Total de páginas: 8


A continuación, se realiza la recolección de los enlaces de cada uno de los apartamentos:

In [5]:
# creación de vector con los enlaces de los apartamentos
enlaces_apartamentos = []
# iteración sobre las páginas
for pagina in range(1, numero_paginas + 1):
    # configuración del seguimiento visual
    progreso = pagina / numero_paginas
    porcentaje = progreso * 100
    barra = '█' * int(porcentaje // 2) + '-' * (50 - int(porcentaje // 2))
    print(f'\rProcesando página {pagina}/{numero_paginas} |{barra}| {porcentaje:.2f}%   ', end='', flush=True)
    if pagina == numero_paginas:
        print()
    
    # definición del enlace de la página
    enlace_pagina = 'https://www.fincaraiz.com.co/arriendo/apartamentos/ibague/tolima/pagina' + str(pagina)
    
    try:
        # solicitud a la página
        respuesta_pagina = requests.get(enlace_pagina, headers=headers, timeout=30)
        respuesta_pagina.raise_for_status()
        arbol_pagina = lxml.html.fromstring(respuesta_pagina.content)
        
        # XPath más robusto y flexible - buscar todos los enlaces de apartamentos
        xpath = '//a[contains(@href, "/apartamento")]'
        enlaces = arbol_pagina.xpath(xpath)
        
        # Si no encuentra nada con el XPath principal, intentar alternativas
        if not enlaces:
            # XPath alternativo más general
            xpath_alt = '//a[contains(@href, "apartamento")]'
            enlaces = arbol_pagina.xpath(xpath_alt)
        
        # Si aún no encuentra nada, intentar buscar por patrones en el texto
        if not enlaces:
            xpath_texto = '//a[contains(text(), "Apartamento") o contains(@href, "ibague")]'
            enlaces = arbol_pagina.xpath(xpath_texto)
        
        # Debugging para ver qué está pasando
        print(f"\nPágina {pagina}: {len(enlaces)} enlaces encontrados")
        
        # procesar los enlaces encontrados
        for enlace in enlaces:
            href = enlace.get('href')
            if href and '/apartamento' in href:
                # asegurar que el enlace sea relativo (sin dominio)
                if href.startswith('/'):
                    enlaces_apartamentos.append(href)
                elif 'fincaraiz.com.co' in href:
                    # extraer solo la parte relativa del enlace
                    href_relativo = href.split('fincaraiz.com.co')[-1]
                    enlaces_apartamentos.append(href_relativo)
                
                # exportación incremental en la carpeta 'datos' en formato parquet
                ruta_parquet = os.path.join('datos', f'{fecha_actual}_enlaces_apartamentos.parquet')
                df_enlaces = pd.DataFrame(enlaces_apartamentos, columns=['enlace'])
                df_enlaces.to_parquet(ruta_parquet, index=False)
    
    except requests.exceptions.RequestException as e:
        print(f"\nError al acceder a la página {pagina}: {e}")
        continue
    except Exception as e:
        print(f"\nError inesperado en página {pagina}: {e}")
        continue

print()  # para salto de línea después de la barra de progreso

# filtra los enlaces que no cumplen con el patrón establecido
pat = re.compile(r"^/apartamento-en-arriendo-[a-z0-9-]*ibague/\d+$")
enlaces_apartamentos = [u for u in enlaces_apartamentos if pat.match(u)]

print(f'Se descargaron {len(enlaces_apartamentos)} enlaces de apartamentos en arriendo.')

Procesando página 1/8 |██████--------------------------------------------| 12.50%   
Página 1: 62 enlaces encontrados
Procesando página 2/8 |████████████--------------------------------------| 25.00%   
Página 2: 63 enlaces encontrados
Procesando página 3/8 |██████████████████--------------------------------| 37.50%   
Página 3: 63 enlaces encontrados
Procesando página 4/8 |█████████████████████████-------------------------| 50.00%   
Página 4: 63 enlaces encontrados
Procesando página 5/8 |███████████████████████████████-------------------| 62.50%   
Página 5: 63 enlaces encontrados
Procesando página 6/8 |█████████████████████████████████████-------------| 75.00%   
Página 6: 63 enlaces encontrados
Procesando página 7/8 |███████████████████████████████████████████-------| 87.50%   
Página 7: 63 enlaces encontrados
Procesando página 8/8 |██████████████████████████████████████████████████| 100.00%   

Página 8: 60 enlaces encontrados

Se descargaron 246 enlaces de apartamentos en arriend

Antes de continuar, se hace la revisión de los enlaces para evitar que se exporten duplicados. Una vez hecha, se hace la exportación:

In [6]:
# eliminación de duplicados en la lista de enlaces
enlaces_apartamentos = list(dict.fromkeys(enlaces_apartamentos))

# exportación de la lista sin duplicados a formato parquet
df_enlaces_unicos = pd.DataFrame(enlaces_apartamentos, columns=['enlace'])
df_enlaces_unicos.to_parquet(f'{fecha_actual}_enlaces_apartamentos.parquet', index=False)

# impresión del resultado para seguimiento
print(f'Se eliminaron duplicados. Ahora hay {len(enlaces_apartamentos)} enlaces únicos.')

Se eliminaron duplicados. Ahora hay 121 enlaces únicos.


De esta forma, se procede a construir la base de datos con la información encontrada en cada enlace:

In [7]:
datos_apartamentos = pd.DataFrame()
ruta_archivo = f'datos/{fecha_actual}_apartamentos_ibague.parquet'

for enlace_apartamento in enlaces_apartamentos:
    # seguimiento visual del progreso en la iteración de los enlaces
    indice_actual = enlaces_apartamentos.index(enlace_apartamento) + 1
    total_enlaces = len(enlaces_apartamentos)
    progreso = indice_actual / total_enlaces
    porcentaje = progreso * 100
    barra = '█' * int(porcentaje // 2) + '-' * (50 - int(porcentaje // 2))
    print(f'\rProcesando enlace {indice_actual}/{total_enlaces} |{barra}| {porcentaje:.2f}%', end='')
    
    # construcción del enlace y solicitud a la página
    enlace_completo = 'https://www.fincaraiz.com.co' + enlace_apartamento
    
    try:
        respuesta_apartamento = requests.get(enlace_completo, headers=headers, timeout=30)
        respuesta_apartamento.raise_for_status()
        arbol_apartamento = lxml.html.fromstring(respuesta_apartamento.content)
    
        # definición de las columnas requeridas
        columnas = [
            'Enlace', 'Precio', 'Baños', 'Área Construida', 'Antigüedad', 'Habitaciones', 'Parqueaderos',
            'Área Privada', 'Estrato', 'Administración', 'Piso N°', 'Acepta mascotas',
            'Documentación requerida', 'Estado', 'Cantidad de pisos', 'Contrato Mínimo',
            'Latitud', 'Longitud'
            ]

        # inicialización de diccionario para almacenar los valores extraídos
        valores = {col: None for col in columnas}
        valores['Enlace'] = enlace_completo

        # extracción del precio (diferentes tipos de xpath)
        precio_apartamento = None
        xpaths_precio = [
            '//span[contains(@class, "price")]//text()',
            '//div[contains(@class, "price")]//text()',
            '//*[contains(text(), "$") and contains(text(), "000")]//text()',
            '//p[contains(text(), "$")]//text()',
            '//span[contains(text(), "$")]//text()'
            ]

        for xpath_precio in xpaths_precio:
            try:
                precio_elements = arbol_apartamento.xpath(xpath_precio)
                if precio_elements:
                    # búsqueda del elemento que contenga un precio válido
                    for precio_elem in precio_elements:
                        if isinstance(precio_elem, str) and '$' in precio_elem and any(char.isdigit() for char in precio_elem):
                            precio_apartamento = precio_elem.strip()
                            break
                    if precio_apartamento:
                        break
            except:
                continue
        
        valores['Precio'] = precio_apartamento

        # extracción de características 
        scripts = arbol_apartamento.xpath('//script[contains(text(), "technicalSheet")]')
        if scripts:
            script_text = scripts[0].text_content().strip()
    
            # Extraer el JSON del script
            json_start = script_text.find('{')
            json_end = script_text.rfind('}') + 1
            json_str = script_text[json_start:json_end]
    
            try:
                data = json.loads(json_str)
                technical_sheet = data['props']['pageProps']['data']['technicalSheet']
        
                for item in technical_sheet:
                    field = item['field']
                    value = item['value']
                    text = item['text']
            

                    # mapeo a nuestras columnas
                    if field == 'bathrooms':
                        valores['Baños'] = value
                    elif field == 'bedrooms':
                        valores['Habitaciones'] = value
                    elif field == 'm2Built':
                        valores['Área Construida'] = value
                    elif field == 'm2apto':
                        valores['Área Privada'] = value
                    elif field == 'stratum':
                        valores['Estrato'] = value
                    elif field == 'floor':
                        valores['Piso N°'] = value
                    elif field == 'constructionYear':
                        valores['Antigüedad'] = value
                    elif field == 'construction_state_name':
                        valores['Estado'] = value
                    elif field == 'garage':
                        valores['Parqueaderos'] = value
                    elif field == 'commonExpenses':
                        valores['Administración'] = value
                    elif field == 'floorsAmount':
                        valores['Cantidad de pisos'] = value
                    elif field == 'allowPets':
                        valores['Acepta mascotas'] = value
                    elif field == 'minimumContract':
                        valores['Contrato Mínimo'] = value
        
                # Extraer coordenadas
                lat = data['props']['pageProps']['data'].get('latitude')
                lon = data['props']['pageProps']['data'].get('longitude')
                if lat:
                    valores['Latitud'] = lat
                if lon:
                    valores['Longitud'] = lon
            
            except (json.JSONDecodeError, KeyError) as e:
                print(f"Error al procesar JSON: {e}")
        
        # adición de la fila al cuadro
        datos_apartamentos = pd.concat([datos_apartamentos, pd.DataFrame([valores])], ignore_index=True)
        
        # Exportación incremental después de cada iteración
        if not datos_apartamentos.empty:
            datos_apartamentos.to_parquet(ruta_archivo, index=False)
        
    except requests.exceptions.RequestException as e:
        print(f"\nError al acceder al enlace {enlace_completo}: {e}")
        continue

print()  # salto de línea después de la barra de progreso

# exportación de la base de datos final
if not datos_apartamentos.empty:
    print(f'Se exportaron {len(datos_apartamentos)} registros de apartamentos.')
    print(f'Archivo guardado como: {ruta_archivo}')
else:
    print('No se encontraron datos para exportar.')    

Procesando enlace 121/121 |██████████████████████████████████████████████████| 100.00%
Se exportaron 121 registros de apartamentos.
Archivo guardado como: datos/20250922_apartamentos_ibague.parquet


Se hace una revisión rápida del resultado:

In [8]:
datos_apartamentos.sample(10, random_state=42)

,Enlace,Precio,Baños,Área Construida,Antigüedad,Habitaciones,Parqueaderos,Área Privada,Estrato,Administración,Piso N°,Acepta mascotas,Documentación requerida,Estado,Cantidad de pisos,Contrato Mínimo,Latitud,Longitud
44,https://www.fincaraiz.com.co/apartamento-en-ar...,$ 3.200.000,4,232 m2,16 a 30 años,5,2,232 m2,4,,7,,None,Usado,7,,4.444440,-75.240016
47,https://www.fincaraiz.com.co/apartamento-en-ar...,$ 600.000,2,60 m2,1 a 8 años,3,1,,2,,10,,None,Usado,,,4.449624,-75.143888
4,https://www.fincaraiz.com.co/apartamento-en-ar...,$ 760.000,1,58 m2,16 a 30 años,2,,58 m2,3,$ 140.000,,,None,Usado,,,4.440667,-75.198977
55,https://www.fincaraiz.com.co/apartamento-en-ar...,$ 4.500.000,3,170 m2,1 a 8 años,4,2,170 m2,4,,11,,None,Usado,,,4.441244,-75.201267
26,https://www.fincaraiz.com.co/apartamento-en-ar...,$ 3.500.000,2,60 m2,1 a 8 años,1,1,56 m2,4,,6,,None,Usado,,,4.438916,-75.198926
64,https://www.fincaraiz.com.co/apartamento-en-ar...,$ 972.000,2,51 m2,1 a 8 años,3,,51 m2,3,$ 189.100,,,None,Usado,,,4.419986,-75.170502
73,https://www.fincaraiz.com.co/apartamento-en-ar...,$ 1.500.000,2,74 m2,1 a 8 años,3,1,,4,,12,,None,Usado,,,4.431127,-75.180289
10,https://www.fincaraiz.com.co/apartamento-en-ar...,$ 562.000,2,44.00 m2,9 a 15 años,3,,44.00 m2,3,"$ 138,000.00",,,None,,,,4.451767,-75.141645
40,https://www.fincaraiz.com.co/apartamento-en-ar...,$ 880.000,2,53 m2,9 a 15 años,3,1,53 m2,3,,7,,None,Usado,,,4.447021,-75.152942
108,https://www.fincaraiz.com.co/apartamento-en-ar...,$ 1.258.000,2,60 m2,16 a 30 años,2,1,60 m2,4,$ 242.000,,,None,Usado,,,4.440267,-75.205204


Se exporta la base resultante:

In [9]:
datos_apartamentos.to_parquet(f'datos/{fecha_actual}_apartamentos_ibague.parquet', index=False)

# Limpieza de datos

Se realiza la limpieza de datos para que correspondan al formato correcto. Inicialmente, se revisa la estructura de la base:

In [10]:
datos_apartamentos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121 entries, 0 to 120
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Enlace                   121 non-null    object 
 1   Precio                   121 non-null    object 
 2   Baños                    121 non-null    object 
 3   Área Construida          121 non-null    object 
 4   Antigüedad               121 non-null    object 
 5   Habitaciones             121 non-null    object 
 6   Parqueaderos             121 non-null    object 
 7   Área Privada             121 non-null    object 
 8   Estrato                  121 non-null    object 
 9   Administración           121 non-null    object 
 10  Piso N°                  121 non-null    object 
 11  Acepta mascotas          121 non-null    object 
 12  Documentación requerida  0 non-null      object 
 13  Estado                   121 non-null    object 
 14  Cantidad de pisos        1

Todos las variables, con la excepción de la latitud y la longitud, aparecen como tipo texto. Ya que hay variables que son de tipo numéricas, se revisa cada una para ver cómo están. Antes de empezar, se eliminan las variables que no aportan información y se estandarizan los valores nulos.

## Eliminación de variables sin información

Se identifican las variables que solo tienen un datos, por lo que no aportan información:

In [11]:
columnas_unico_valor = [col for col in datos_apartamentos.columns if datos_apartamentos[col].nunique(dropna=False) == 1]
datos_apartamentos[columnas_unico_valor]

,Acepta mascotas,Documentación requerida,Contrato Mínimo
0,,None,
1,,None,
2,,None,
3,,None,
4,,None,
...,...,...,...
116,,None,
117,,None,
118,,None,
119,,None,


De esta forma, se eliminan:

In [12]:
datos_apartamentos = datos_apartamentos.drop(columns=columnas_unico_valor)

## Valores nulos

Aquellos valores con valor nulo se estandarizan para que tengan una misma forma.

In [13]:
datos_apartamentos = datos_apartamentos.replace(['¡Pregúntale!', '', 'None', '...', '<NA>'], pd.NA)

## Identificador

Se construye un identificador a partir del enlace del apartamento:

In [14]:
# Extraer el identificador del enlace después del último "/"
datos_apartamentos['Identificador'] = datos_apartamentos['Enlace'].str.split('/').str[-1]

# muestra de 10 filas con las columnas 'Enlace' e 'Identificador'
datos_apartamentos[['Enlace', 'Identificador']].sample(10, random_state=42)

,Enlace,Identificador
44,https://www.fincaraiz.com.co/apartamento-en-ar...,191180666
47,https://www.fincaraiz.com.co/apartamento-en-ar...,192859122
4,https://www.fincaraiz.com.co/apartamento-en-ar...,192755337
55,https://www.fincaraiz.com.co/apartamento-en-ar...,192829838
26,https://www.fincaraiz.com.co/apartamento-en-ar...,192371808
64,https://www.fincaraiz.com.co/apartamento-en-ar...,192861063
73,https://www.fincaraiz.com.co/apartamento-en-ar...,192884964
10,https://www.fincaraiz.com.co/apartamento-en-ar...,191344849
40,https://www.fincaraiz.com.co/apartamento-en-ar...,192669238
108,https://www.fincaraiz.com.co/apartamento-en-ar...,192712224


Se elimina la variable original:

In [15]:
datos_apartamentos = datos_apartamentos.drop(columns=['Enlace'])

## Baños

Antes de convertir la variable, se revisan algunos casos al azar buscando identificar características generales:

In [16]:
datos_apartamentos[['Baños']].sample(10, random_state=42)

,Baños
44,4
47,2
4,1
55,3
26,2
64,2
73,2
10,2
40,2
108,2


Se realiza la conversión:

In [17]:
datos_apartamentos['Baños'] = pd.to_numeric(datos_apartamentos['Baños'], errors='coerce').astype('Int64')

Se revisa el comportamiento de la variable:

In [18]:
datos_apartamentos[['Baños']].describe()

,Baños
count,120.0
mean,1.958333
std,0.737881
min,1.0
25%,2.0
50%,2.0
75%,2.0
max,5.0


Como no se ve nada extraño, se deja como está.

## Áreas

Antes de convertir las variables, se revisan algunos casos al azar buscando identificar algo raro:

In [19]:
datos_apartamentos[['Área Construida', 'Área Privada']].sample(10, random_state=42)

,Área Construida,Área Privada
44,232 m2,232 m2
47,60 m2,<NA>
4,58 m2,58 m2
55,170 m2,170 m2
26,60 m2,56 m2
64,51 m2,51 m2
73,74 m2,<NA>
10,44.00 m2,44.00 m2
40,53 m2,53 m2
108,60 m2,60 m2


Como en este caso se especifica la unidad de medida, ésta se incluye en el nombre de las variables y se convierten a numéricas luego de eliminar la parte de texto.

In [20]:
datos_apartamentos['Área Construida (metros cuadrados)'] = datos_apartamentos['Área Construida'].str.extract(r'([\d\.,]+)')[0].str.replace(',', '').astype(float)
datos_apartamentos['Área Privada (metros cuadrados)'] = datos_apartamentos['Área Privada'].str.extract(r'([\d\.,]+)')[0].str.replace(',', '').astype(float)

Se revisa el comportamiento de las variables:

In [21]:
datos_apartamentos[['Área Construida (metros cuadrados)', 'Área Privada (metros cuadrados)']].describe()

,Área Construida (metros cuadrados),Área Privada (metros cuadrados)
count,121.000000,92.000000
mean,193.898182,78.554348
std,901.158274,51.031204
min,1.000000,1.000000
25%,56.000000,52.750000
50%,65.000000,60.000000
75%,90.000000,80.000000
max,8450.000000,300.000000


No parece haber nada raro, por lo que se eliminan las variables originales:

In [22]:
datos_apartamentos = datos_apartamentos.drop(columns=['Área Construida', 'Área Privada'])

## Habitaciones

Antes de convertir la variable, se revisan algunos casos al azar buscando identificar características generales:

In [23]:
datos_apartamentos[['Habitaciones']].sample(10, random_state=42)

,Habitaciones
44,5
47,3
4,2
55,4
26,1
64,3
73,3
10,3
40,3
108,2


Se realiza la conversión:

In [24]:
datos_apartamentos['Habitaciones'] = pd.to_numeric(datos_apartamentos['Habitaciones'], errors='coerce').astype('Int64')

Se revisa el comportamiento de la variable:

In [25]:
datos_apartamentos[['Habitaciones']].describe()

,Habitaciones
count,120.0
mean,2.8
std,0.836158
min,1.0
25%,2.0
50%,3.0
75%,3.0
max,8.0


Como no hay nada raro, se deja así.

## Parqueaderos

Antes de convertir la variable, se revisan algunos casos al azar buscando identificar características generales:

In [26]:
datos_apartamentos[['Parqueaderos']].sample(10, random_state=42)

,Parqueaderos
44,2
47,1
4,<NA>
55,2
26,1
64,<NA>
73,1
10,<NA>
40,1
108,1


Se realiza la conversión:

In [27]:
datos_apartamentos['Parqueaderos'] = pd.to_numeric(datos_apartamentos['Parqueaderos'], errors='coerce').astype('Int64')

Se revisa el comportamiento de la variable:

In [28]:
datos_apartamentos[['Parqueaderos']].describe()

,Parqueaderos
count,81.0
mean,1.17284
std,0.542912
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,5.0


## Administración

Antes de hacer cualquier conversión, se revisan algunos casos al azar buscando identificar características generales:

In [29]:
datos_apartamentos[['Administración']].dropna().sample(10, random_state=42)

,Administración
36,$ 153.000
109,$ 380.000
114,$ 187.591
108,$ 242.000
51,$ 227.000
80,$ 233.200
78,$ 1.040.000
77,$ 250.000
92,$ 206.400
62,$ 291.693


En este caso, se trata de un valor monetario en formato de número latinoamericano (coma como separador decimal y punto como separador de miles) y con signo monetario. Sin embargo, hay casos en que el valor fue puesto en formato anglosajón, por lo que debe tomarse en consideración. De esta forma, se procede a limpiar estas características antes de convertir la variable a numérica:

In [30]:
# eliminación de espacios
datos_apartamentos['Administración'] = datos_apartamentos['Administración'].str.replace(' ', '', regex=False)
# eliminación del signo monetario
datos_apartamentos['Administración'] = datos_apartamentos['Administración'].str.replace('$', '', regex=False)
# identificación de números con formato latinoamericano
datos_apartamentos['Formato ESP'] = datos_apartamentos['Administración'].str.extract(r'^(\d{1,3}\.\d{3}(?:,\d+)?$)', expand=False).notna()
# identificación de números con formato anglosajón
datos_apartamentos['Formato ENG'] = datos_apartamentos['Administración'].str.extract(r'^(\d{1,3},\d{3}(?:\.\d+)?$)', expand=False).notna()
# aplicación de la normalización del formato
datos_apartamentos.loc[datos_apartamentos['Formato ESP'], 'Administración'] = (
    datos_apartamentos.loc[datos_apartamentos['Formato ESP'], 'Administración']
    .str.replace('.', '', regex=False)
    .str.replace(',', '.', regex=False)
)
datos_apartamentos.loc[datos_apartamentos['Formato ENG'], 'Administración'] = (
    datos_apartamentos.loc[datos_apartamentos['Formato ENG'], 'Administración']
    .str.replace(',', '', regex=False)
)
# conversion de nulos
datos_apartamentos['Administración'] = datos_apartamentos['Administración'].replace('<NA>', pd.NA)
# conversión de la variable a numérica
datos_apartamentos['Administración'] = pd.to_numeric(datos_apartamentos['Administración'], errors='coerce')
# eliminación de las variables auxiliares
datos_apartamentos = datos_apartamentos.drop(columns=['Formato ESP', 'Formato ENG'])
# revisión de casos
datos_apartamentos[['Administración']].dropna().sample(10, random_state=42)

,Administración
81,220000.0
102,500000.0
80,233200.0
109,380000.0
69,97300.0
99,210000.0
35,281000.0
62,291693.0
10,138000.0
77,250000.0


## Número del piso

En el caso del número de piso, debido a que pisos altos pueden ser más o menos atractivos según el contexto, se convierte la variable a numérica:

In [31]:
datos_apartamentos['Piso N°'] = pd.to_numeric(datos_apartamentos['Piso N°'], errors='coerce').astype('Int64')

Se revisa que haya quedado bien:

In [32]:
datos_apartamentos[['Piso N°']].dropna().sample(10, random_state=42)

,Piso N°
60,10
20,12
63,5
31,15
41,1
113,2
86,15
53,6
89,4
29,8


## Cantidad de pisos

De forma similar a la variable anterior, se convierte a numérica:

In [33]:
datos_apartamentos['Cantidad de pisos'] = pd.to_numeric(datos_apartamentos['Cantidad de pisos'], errors='coerce').astype('Int64')

Se revisa que haya quedado bien:

In [34]:
datos_apartamentos[['Cantidad de pisos']].describe()

,Cantidad de pisos
count,8.0
mean,6.0
std,4.105745
min,3.0
25%,3.0
50%,4.5
75%,7.25
max,15.0


## Administración

Antes de hacer cualquier conversión, se revisan algunos casos al azar buscando identificar características generales:

In [35]:
datos_apartamentos[['Precio']].dropna().sample(10, random_state=42)

,Precio
44,$ 3.200.000
47,$ 600.000
4,$ 760.000
55,$ 4.500.000
26,$ 3.500.000
64,$ 972.000
73,$ 1.500.000
10,$ 562.000
40,$ 880.000
108,$ 1.258.000


En este caso, se trata de un valor monetario en formato de número latinoamericano (coma como separador decimal y punto como separador de miles) y con signo monetario. Sin embargo, hay casos en que el valor fue puesto en formato anglosajón, por lo que debe tomarse en consideración. De esta forma, se procede a limpiar estas características antes de convertir la variable a numérica:

In [36]:
# eliminación de espacios
datos_apartamentos['Precio'] = datos_apartamentos['Precio'].str.replace(' ', '', regex=False)
# eliminación del signo monetario
datos_apartamentos['Precio'] = datos_apartamentos['Precio'].str.replace('$', '', regex=False)
# identificación de números con formato latinoamericano
# Generalización para detectar números en formato latinoamericano con millones, miles y decimales
datos_apartamentos['Formato ESP'] = datos_apartamentos['Precio'].str.extract(r'^(\d{1,3}(?:\.\d{3})+(?:,\d+)?|\d+(?:,\d+)?$)', expand=False).notna()
# identificación de números con formato anglosajón
# Generalización para detectar números en formato anglosajón con millones, miles y decimales
datos_apartamentos['Formato ENG'] = datos_apartamentos['Precio'].str.extract(r'^(\d{1,3}(?:,\d{3})+(?:\.\d+)?|\d+(?:\.\d+)?$)', expand=False).notna()
# aplicación de la normalización del formato
datos_apartamentos.loc[datos_apartamentos['Formato ESP'], 'Precio'] = (
    datos_apartamentos.loc[datos_apartamentos['Formato ESP'], 'Precio']
    .str.replace('.', '', regex=False)
    .str.replace(',', '.', regex=False)
)
datos_apartamentos.loc[datos_apartamentos['Formato ENG'], 'Precio'] = (
    datos_apartamentos.loc[datos_apartamentos['Formato ENG'], 'Precio']
    .str.replace(',', '', regex=False)
)
# conversion de nulos
datos_apartamentos['Precio'] = datos_apartamentos['Precio'].replace('<NA>', pd.NA)
# conversión de la variable a numérica
datos_apartamentos['Precio'] = pd.to_numeric(datos_apartamentos['Precio'], errors='coerce')
# eliminación de las variables auxiliares
datos_apartamentos = datos_apartamentos.drop(columns=['Formato ESP', 'Formato ENG'])
# revisión de casos
datos_apartamentos[['Precio']].dropna().sample(10, random_state=42)

,Precio
44,3200000
47,600000
4,760000
55,4500000
26,3500000
64,972000
73,1500000
10,562000
40,880000
108,1258000


# Exportación de la base final

In [37]:
datos_apartamentos[[
    'Identificador', 'Área Construida (metros cuadrados)', 'Área Privada (metros cuadrados)',
    'Habitaciones', 'Baños', 'Estrato', 'Parqueaderos', 'Antigüedad', 'Administración', 'Piso N°',
    'Estado', 'Cantidad de pisos', 'Latitud', 'Longitud', 'Precio'
    ]].to_parquet(f"datos/{fecha_actual}_apartamentos_ibague.parquet", index=False)